## Load necessary modules

In [1]:
%cd ~/mlflow_dev_env/keras-retinanet/

/home/ubuntu/mlflow_dev_env/keras-retinanet


In [2]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
from tensorflow import keras
import tensorflow as tf

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import mlflow

2022-06-06 15:57:00.726140: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


## Load RetinaNet model

In [4]:
print('Loading mlflow model, this may take a second...')
tracking_uri = "http://0.0.0.0:5000"
model_uri = 'runs:/4bc86b9f457e4e4387e8506bd7c9db30/WeedKerasRetinaNet'

mlflow.set_tracking_uri(tracking_uri)
model = mlflow.keras.load_model(model_uri)

# load label to names mapping for visualization purposes
labels_to_names = {0: 'weed', 1: 'wheat'}

Loading mlflow model, this may take a second...


## Run detection on example

In [5]:
%cd ~/mlflow_dev_env/training_data/

/home/ubuntu/mlflow_dev_env/training_data


In [6]:
import glob

files = glob.glob('psuedo_labels/*.jpg')
print(len(files))

10000


In [7]:
import random

random_sample = random.sample(files, 2000)
print(len(random_sample))

2000


In [8]:
import cv2

In [9]:
from ensemble_boxes import *
import imutils
def run_wbf(boxes, scores, labels, image_size=[1333,666], iou_thr=0.5, skip_box_thr=0.7, weights=None):
    divide_box = np.array(image_size + image_size)
    boxes = [box / divide_box for box in boxes]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes * divide_box
    return boxes, scores, labels

def TTAImage(image, index):
    image1 = image.copy()
    if index==0: 
        rotated_image = imutils.rotate_bound(image1, 90)
        return rotated_image
    elif index==1:
        rotated_image2 = imutils.rotate_bound(image1, 180)
        return rotated_image2
    elif index==2:
        rotated_image3 = imutils.rotate_bound(image1, 270)
        return rotated_image3
    elif index == 3:
        return image1
    
def rotBoxes90(boxes, im_w, im_h):
    ret_boxes =[]
    for box in boxes:
        box = box.tolist()
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = x1-im_w//2, im_h//2 - y1, x2-im_w//2, im_h//2 - y2
        x1, y1, x2, y2 = y1, -x1, y2, -x2
        x1, y1, x2, y2 = int(x1+im_h//2), int(im_w//2 - y1), int(x2+im_h//2), int(im_w//2 - y2)
        x1a, y1a, x2a, y2a = min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)
        ret_boxes.append([x1a, y1a, x2a, y2a])
    return np.array(ret_boxes)

In [10]:
def show_pred(boxes, scores, labels, img):
    i = 0
    img_c = img.copy()
    
    for box, score, label in zip(boxes, scores, labels):
         # scores are sorted so we can break
        if score < 0.2 and label == 1:
            break
        if score < 0.3 and label == 0:
            continue
        if label == 0:
            class_name = "weed"
        else:
            class_name = "wheat"
        color = label_color(int(label))
        
        cv2.rectangle(img_c, (int(box[0]),int(box[1])), (int(box[2]),int(box[3])), color, 2)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, box, caption)
        i += 1
    plt.clf()
    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(img_c)
    
    plt.show()
    

In [11]:
%ls

classes.csv           test/                     validation_annotations.csv
psuedo_label_sample/  train/
psuedo_labels/        training_annotations.csv


In [12]:
import json
import shutil
# load image
total_size = len(files)
anno_id = 0
j = 0
for image_file in random_sample:
    #image_file = str(2) + ".jpg"
    image = read_image_bgr(image_file)
    jpg_file = image_file.split("psuedo_labels/")[1]
    image_folder_file = image_file
    sample_folder_file = 'psuedo_label_sample/' + jpg_file
    shutil.copy(image_folder_file, sample_folder_file)
    image_json = jpg_file.split(".jpg")[0] + ".json"
    

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    draw_w, draw_h = draw.shape[:2]
    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)
    im_w, im_h = image.shape[:2]
    # process image
    start = time.time()
    #boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))

    enboxes = []
    enscores = []
    enlabels = []
    for i in range(4):
        im0 = TTAImage(image, i)
        im_c = TTAImage(draw, i)

        boxes, scores, labels = model.predict_on_batch(np.expand_dims(im0, axis=0))
        boxes /= scale
        boxes = boxes[0]
        scores = scores[0]
        labels = labels[0]
        #show_pred(boxes, scores, labels, im_c)

        for _ in range(3-i):
            h, w = im_c.shape[:2]
            im_c = imutils.rotate_bound(im_c, 90)
            boxes = rotBoxes90(boxes, w, h)
        enboxes.append(boxes)
        enscores.append(scores)
        enlabels.append(labels)


    boxes, scores, labels = run_wbf(enboxes, enscores, enlabels, iou_thr=0.6, skip_box_thr=0.25)

    # correct for image scale
    #boxes /= scale

    label_list = []
    # visualize detections
    boxes = boxes.astype(int).tolist()
    scores = scores.tolist()
    labels = labels.astype(int).tolist()

    #show_pred(boxes, scores, labels, draw)
    for box, score, label in zip(boxes, scores, labels):
         # scores are sorted so we can break
        if score < 0.2 and label == 1:
            break
        if score < 0.3 and label == 0:
            continue
        if label == 0:
            class_name = "weed"
        else:
            class_name = "wheat"
            
        label_dict = {"id": anno_id,
                      "x1": int(box[0]), "x2": int(box[2]), "y1": int(box[1]), "y2": int(box[3]), 
                      "class_name": class_name}
        label_list.append(label_dict)
        anno_id +=1

    img_dict = {"image_name": image_file,
                "image_height": 600, "image_width": 1200, 
                "labels": label_list}
    print("Files Left: " + str(total_size - j))
    j -= 1

    with open("psuedo_label_sample/" + image_json, 'w') as outfile:
        json.dump(img_dict, outfile)

2022-06-06 15:57:42.711242: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-06 15:57:42.748975: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2499995000 Hz
2022-06-06 15:57:43.654606: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-06-06 15:57:45.745549: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-06-06 15:57:46.252804: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


Files Left: 10000
Files Left: 10001
Files Left: 10002
Files Left: 10003
Files Left: 10004
Files Left: 10005
Files Left: 10006
Files Left: 10007
Files Left: 10008
Files Left: 10009
Files Left: 10010
Files Left: 10011
Files Left: 10012
Files Left: 10013
Files Left: 10014
Files Left: 10015
Files Left: 10016
Files Left: 10017
Files Left: 10018
Files Left: 10019
Files Left: 10020
Files Left: 10021
Files Left: 10022
Files Left: 10023
Files Left: 10024
Files Left: 10025
Files Left: 10026
Files Left: 10027
Files Left: 10028
Files Left: 10029
Files Left: 10030
Files Left: 10031
Files Left: 10032
Files Left: 10033
Files Left: 10034
Files Left: 10035
Files Left: 10036
Files Left: 10037
Files Left: 10038
Files Left: 10039
Files Left: 10040
Files Left: 10041
Files Left: 10042
Files Left: 10043
Files Left: 10044
Files Left: 10045
Files Left: 10046
Files Left: 10047
Files Left: 10048
Files Left: 10049
Files Left: 10050
Files Left: 10051
Files Left: 10052
Files Left: 10053
Files Left: 10054
Files Left

Files Left: 10456
Files Left: 10457
Files Left: 10458
Files Left: 10459
Files Left: 10460
Files Left: 10461
Files Left: 10462
Files Left: 10463
Files Left: 10464
Files Left: 10465
Files Left: 10466
Files Left: 10467
Files Left: 10468
Files Left: 10469
Files Left: 10470
Files Left: 10471
Files Left: 10472
Files Left: 10473
Files Left: 10474
Files Left: 10475
Files Left: 10476
Files Left: 10477
Files Left: 10478
Files Left: 10479
Files Left: 10480
Files Left: 10481
Files Left: 10482
Files Left: 10483
Files Left: 10484
Files Left: 10485
Files Left: 10486
Files Left: 10487
Files Left: 10488
Files Left: 10489
Files Left: 10490
Files Left: 10491
Files Left: 10492
Files Left: 10493
Files Left: 10494
Files Left: 10495
Files Left: 10496
Files Left: 10497
Files Left: 10498
Files Left: 10499
Files Left: 10500
Files Left: 10501
Files Left: 10502
Files Left: 10503
Files Left: 10504
Files Left: 10505
Files Left: 10506
Files Left: 10507
Files Left: 10508
Files Left: 10509
Files Left: 10510
Files Left

Files Left: 10912
Files Left: 10913
Files Left: 10914
Files Left: 10915
Files Left: 10916
Files Left: 10917
Files Left: 10918
Files Left: 10919
Files Left: 10920
Files Left: 10921
Files Left: 10922
Files Left: 10923
Files Left: 10924
Files Left: 10925
Files Left: 10926
Files Left: 10927
Files Left: 10928
Files Left: 10929
Files Left: 10930
Files Left: 10931
Files Left: 10932
Files Left: 10933
Files Left: 10934
Files Left: 10935
Files Left: 10936
Files Left: 10937
Files Left: 10938
Files Left: 10939
Files Left: 10940
Files Left: 10941
Files Left: 10942
Files Left: 10943
Files Left: 10944
Files Left: 10945
Files Left: 10946
Files Left: 10947
Files Left: 10948
Files Left: 10949
Files Left: 10950
Files Left: 10951
Files Left: 10952
Files Left: 10953
Files Left: 10954
Files Left: 10955
Files Left: 10956
Files Left: 10957
Files Left: 10958
Files Left: 10959
Files Left: 10960
Files Left: 10961
Files Left: 10962
Files Left: 10963
Files Left: 10964
Files Left: 10965
Files Left: 10966
Files Left

Files Left: 11368
Files Left: 11369
Files Left: 11370
Files Left: 11371
Files Left: 11372
Files Left: 11373
Files Left: 11374
Files Left: 11375
Files Left: 11376
Files Left: 11377
Files Left: 11378
Files Left: 11379
Files Left: 11380
Files Left: 11381
Files Left: 11382
Files Left: 11383
Files Left: 11384
Files Left: 11385
Files Left: 11386
Files Left: 11387
Files Left: 11388
Files Left: 11389
Files Left: 11390
Files Left: 11391
Files Left: 11392
Files Left: 11393
Files Left: 11394
Files Left: 11395
Files Left: 11396
Files Left: 11397
Files Left: 11398
Files Left: 11399
Files Left: 11400
Files Left: 11401
Files Left: 11402
Files Left: 11403
Files Left: 11404
Files Left: 11405
Files Left: 11406
Files Left: 11407
Files Left: 11408
Files Left: 11409
Files Left: 11410
Files Left: 11411
Files Left: 11412
Files Left: 11413
Files Left: 11414
Files Left: 11415
Files Left: 11416
Files Left: 11417
Files Left: 11418
Files Left: 11419
Files Left: 11420
Files Left: 11421
Files Left: 11422
Files Left

Files Left: 11824
Files Left: 11825
Files Left: 11826
Files Left: 11827
Files Left: 11828
Files Left: 11829
Files Left: 11830
Files Left: 11831
Files Left: 11832
Files Left: 11833
Files Left: 11834
Files Left: 11835
Files Left: 11836
Files Left: 11837
Files Left: 11838
Files Left: 11839
Files Left: 11840
Files Left: 11841
Files Left: 11842
Files Left: 11843
Files Left: 11844
Files Left: 11845
Files Left: 11846
Files Left: 11847
Files Left: 11848
Files Left: 11849
Files Left: 11850
Files Left: 11851
Files Left: 11852
Files Left: 11853
Files Left: 11854
Files Left: 11855
Files Left: 11856
Files Left: 11857
Files Left: 11858
Files Left: 11859
Files Left: 11860
Files Left: 11861
Files Left: 11862
Files Left: 11863
Files Left: 11864
Files Left: 11865
Files Left: 11866
Files Left: 11867
Files Left: 11868
Files Left: 11869
Files Left: 11870
Files Left: 11871
Files Left: 11872
Files Left: 11873
Files Left: 11874
Files Left: 11875
Files Left: 11876
Files Left: 11877
Files Left: 11878
Files Left

KeyboardInterrupt: 